In [ ]:
!python phase6_anomaly_scoring.py --data-dir data/model_ready --checkpoint-path checkpoints/hvae_v2/best_model.pt --split test --output-dir data/anomaly_scores --batch-size 4096 --device cuda


[INFO] Using device: cuda
[INFO] Loading Phase 3 artifacts from: data/model_ready\phase3_artifacts.pkl
[INFO] Category sizes for model reconstruction:
  num_start_stations: 1807
  num_end_stations  : 1802
  num_ride_types    : 4
  num_member_types  : 3
  num_numeric_features: 10
[INFO] Loading model checkpoint from: checkpoints/hvae_v2/best_model.pt
[INFO] Building anomaly dataset for split='test'...
[INFO] Computing anomaly scores on 297,877 samples...
  [INFO] Processed 204,800 samples...

ANOMALY SCORE SUMMARY (test split)
count    297877.000000
mean         -1.039350
std          21.731602
min          -5.626651
50%          -1.601471
90%          -0.902468
95%          -0.273548
99%           3.682961
max        8967.341797
Name: anomaly_score, dtype: float64

[TOP 10 MOST ANOMALOUS TRIPS]
                 ride_id       started_at_parsed  ... anomaly_score anomaly_rank
171004  7E66493C044A73FE 2025-03-24 23:23:26.152  ...   8967.341797            1
76658   6B562428E92AB639 2025-03

c:\ECE 5831 - LOCAL\ece-final-project\phase6_anomaly_scoring.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_path, map_location=device)


In [ ]:
!python phase7_latent_analysis.py --data-dir data/model_ready --anomaly-path data/anomaly_scores/anomaly_scores_test.parquet --checkpoint-path checkpoints/hvae_v2/best_model.pt --split test --output-dir data/latent_analysis --num-clusters 8 --batch-size 4096 --device cuda


[INFO] Using device: cuda
[INFO] Loading Phase 3 artifacts from: data/model_ready\phase3_artifacts.pkl
[INFO] Category sizes for model reconstruction:
  num_start_stations: 1807
  num_end_stations  : 1802
  num_ride_types    : 4
  num_member_types  : 3
  num_numeric_features: 10
[INFO] Loading model checkpoint from: checkpoints/hvae_v2/best_model.pt
[INFO] Building latent dataset from: data/anomaly_scores/anomaly_scores_test.parquet
[INFO] Extracting latents for 297,877 samples...
  [INFO] Processed 204,800 samples...
[INFO] Latent shapes: z_global=(297877, 16), z_individual=(297877, 16)
[INFO] Running k-means with K=8 on z_global...

CLUSTER SUMMARY (split=test, K=8)

[Cluster sizes]
intent_cluster
0    61219
1    40764
2    43541
3    22789
4    55084
5     9656
6     3696
7    61128
Name: count, dtype: int64

[Cluster: duration & anomaly stats]
                n_trips  mean_duration  median_duration  mean_anom    p95_anom
intent_cluster                                               

c:\ECE 5831 - LOCAL\ece-final-project\phase7_latent_analysis.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_path, map_location=device)


In [2]:
!python phase8_case_studies.py --input-path data/latent_analysis/latent_analysis_test.parquet --split test --output-dir data/case_studies --top-k-global 200 --top-k-per-cluster 50 --p95-thresh 0.95 --p99-thresh 0.99


[INFO] Loading latent+anomaly data from: data/latent_analysis/latent_analysis_test.parquet
[INFO] Selecting global top-200 anomalies...
[INFO] Saved global top-200 anomalies to:
       data/case_studies\global_top200_anomalies_test.parquet
       data/case_studies\global_top200_anomalies_test.csv
[INFO] Selecting top-50 anomalies per cluster...
[INFO] Saved per-cluster top-50 anomalies to:
       data/case_studies\cluster_top50_anomalies_test.parquet
       data/case_studies\cluster_top50_anomalies_test.csv
[INFO] Computing threshold-based anomaly flags...
  [STATS] p95 threshold (top 5%): -0.2735
          Count >= p95: 14894 (5.00 % of trips)
  [STATS] p99 threshold (top 1%): 3.6830
          Count >= p99: 2979 (1.00 % of trips)
[INFO] Saved latent+flags dataframe to: data/case_studies\latent_with_flags_test.parquet

[Cluster-level anomaly counts (p99 flag)]
                n_anom_p99  n_total  pct_anom_p99
intent_cluster                                   
0                        2 

In [3]:
!python phase9_plots_and_results.py

[INFO] Using device: cuda
[INFO] Loading Phase 3 artifacts from: data/model_ready\phase3_artifacts.pkl
[INFO] Loading model checkpoint from: checkpoints/hvae_v2/best_model.pt

=== Rideable-type classification (test) ===

[RESULT] Test rideable_type accuracy: 84.70%

[CONFUSION MATRIX (raw counts)]
[[     0      0      0      0]
 [     0  94324  16427      0]
 [     0  29132 157992      2]
 [     0      0      0      0]]

[CLASSIFICATION REPORT]
                  precision    recall  f1-score   support

           <UNK>       0.00      0.00      0.00         0
    classic_bike       0.76      0.85      0.81    110751
   electric_bike       0.91      0.84      0.87    187126
electric_scooter       0.00      0.00      0.00         0

        accuracy                           0.85    297877
       macro avg       0.42      0.42      0.42    297877
    weighted avg       0.85      0.85      0.85    297877

[INFO] Saved confusion matrix figure to: figures\fig_rideable_confusion_test.png

==

c:\ECE 5831 - LOCAL\ece-final-project\phase9_plots_and_results.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_path, map_location=device)
c